In [1]:
import pandas as pd
import numpy as np
from settings import Config
from mysql_db import Database
import pdcast as pdc
import s3_upload_download as s3con
import os

# NOTEBOOK DESCRIPTION: 

Perform different checks on the transferred tables:
1. Same number of rows as in the originals?
2. Duplicates or nans?
3. Does the data only contain policies signed after 2015?

The POLICIES table (polite_rca) only contains policies signed after 2015. Performing a MIN() operation of the policy id column indicates 2230446 to be the earlierst policy. 

In [2]:
# initialise the s3_connector object needed to read/write files into an S3 bucket
s3con = s3con.s3_connector()

## a. Attributes

### Attributes 1 to 5 were created from the CLIENTS table (clienti), with an original count of 55564938.
However, this must be halved to account for the insured/user duplicates, thus 27782469.

In [3]:
# check att1_type
df = s3con.read('att1_type.feather')
print('Number of rows is', df.shape[0])
print('Number of complete duplicates is', df.duplicated().sum())
print('Number of POLICY ID (idPolita) duplicates is', df.duplicated('idPolita').sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27782469
Number of complete duplicates is 0
Number of POLICY ID (idPolita) duplicates is 0
Number of nans is 
 idPolita    0
tip         0
dtype: int64


In [35]:
df[df.idPolita < 2230446]

,idPolita,tip


Since attributes 1 to 5 all come from the same original table, there is no need to check whether each attribute table contains policies from 2015 onwards.

In [36]:
# check att2_age
df = s3con.read('att2_age.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27782469
Number of duplicates is 0
Number of nans is 
 idPolita    0
varsta      0
dtype: int64


In [37]:
# check att3_judet
df = s3con.read('att3_judet.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27782469
Number of duplicates is 0
Number of nans is 
 idPolita    0
judet       0
dtype: int64


In [40]:
# check att4_localitate
df = s3con.read('att4_localitate.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27782469
Number of duplicates is 0
Number of nans is 
 idPolita      0
localitate    0
dtype: int64


In [41]:
# check att5_sex
df = s3con.read('att5_sex.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27782469
Number of duplicates is 0
Number of nans is 
 idPolita    0
sex         0
dtype: int64


Attributes 1 to 5 do not appear to have obvious issues.

### Attributes 6 to 12 were created from the OBJECTS table (obiecte), with a count of 27598519.

In [19]:
# check att6_category_type.feather
df = s3con.read('att6_category_type_a.feather')
df2 = s3con.read('att6_category_type_b.feather')

con = pd.concat([df, df2], axis= 0, ignore_index= True)
con.shape[0]

27598520

In [20]:
con.duplicated().sum()

1

In [21]:
con.drop_duplicates(inplace = True, ignore_index = True)

In [23]:
con.isnull().sum()

idPolita      0
categorie     0
tipVehicul    0
dtype: int64

In [24]:
# save to S3
con.to_feather('att6_category_type.feather')
s3con.write('att6_category_type.feather')
os.remove('att6_category_type.feather')

In [43]:
df = s3con.read('att6_category_type.feather')
df[df.idPolita < 2230446]

,idPolita,categorie,tipVehicul


Again, since attributes 6 to 12 come from the same table, no need to continue checking whether all policies are 2015 onwards.

In [31]:
# check att7_category_type.feather
df = s3con.read('att7_make_a.feather')
df2 = s3con.read('att7_make_b.feather')

con = pd.concat([df, df2], axis= 0, ignore_index= True)
con.shape[0]

27598520

In [34]:
con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598520 entries, 0 to 27598519
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   idPolita  uint32
 1   marca     object
 2   model     object
dtypes: object(2), uint32(1)
memory usage: 526.4+ MB


In [35]:
con = pdc.downcast(con)
con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598520 entries, 0 to 27598519
Data columns (total 3 columns):
 #   Column    Dtype   
---  ------    -----   
 0   idPolita  uint32  
 1   marca     category
 2   model     category
dtypes: category(2), uint32(1)
memory usage: 274.9 MB


In [36]:
con.drop_duplicates(inplace = True, ignore_index = True)

In [37]:
con.isnull().sum()

idPolita    0
marca       0
model       0
dtype: int64

In [38]:
# save to S3
con.to_feather('att7_make.feather')
s3con.write('att7_make.feather')
os.remove('att7_make.feather')

In [5]:
# check att8_capacity
df = s3con.read('att8_capacity.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of POLICY ID (idPolita) duplicates is', df.duplicated('idPolita').sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598519
Number of duplicates is 0
Number of POLICY ID (idPolita) duplicates is 0
Number of nans is 
 idPolita      0
capacitate    0
dtype: int64


In [45]:
# check att9_power
df = s3con.read('att9_power.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598519
Number of duplicates is 0
Number of nans is 
 idPolita    0
putere      0
dtype: int64


In [47]:
# check att10_seats
df = s3con.read('att10_seats.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598519
Number of duplicates is 0
Number of nans is 
 idPolita    0
locuri      0
dtype: int64


In [48]:
# check att11_year
df = s3con.read('att11_year.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598519
Number of duplicates is 0
Number of nans is 
 idPolita        0
anFabricatie    0
dtype: int64


In [49]:
# check att12_weight
df = s3con.read('att12_weight.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598519
Number of duplicates is 0
Number of nans is 
 idPolita       0
masaTehnica    0
dtype: int64


### Attributes 13 and 14 were created from the POLICIES table (polite), with a count of 27598532.

In [6]:
# check att13_bonus_malus
df = s3con.read('att13_bonus_malus.feather')
print('Number of rows is', df.shape[0])
print('Number of complete duplicates is', df.duplicated().sum())
print('Number of POLICY ID (idPolita) duplicates is', df.duplicated('idPolita').sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598532
Number of complete duplicates is 0
Number of POLICY ID (idPolita) duplicates is 0
Number of nans is 
 idPolita    0
clasaBM     0
dtype: int64


In [51]:
df[df.idPolita < 2230446]

,idPolita,clasaBM


In [52]:
# check att14_no_rates
df = s3con.read('att14_no_rates.feather')
print('Number of rows is', df.shape[0])
print('Number of duplicates is', df.duplicated().sum())
print('Number of nans is \n', df.isnull().sum())

Number of rows is 27598532
Number of duplicates is 0
Number of nans is 
 idPolita    0
nrRate      0
dtype: int64


In [53]:
df[df.idPolita < 2230446]

,idPolita,nrRate


## b. Exposures and number of events

#### Events_no was created from the CLAIMS table (daune), where the count of distinct policies was 624963.

In [7]:
df = s3con.read('events_no.feather')
df.head()

,idPolita,total_ev,ev_2015,ev_2016,ev_2017,ev_2018,ev_2019,ev_2020,ev_2021
0,2230466,1,1,0,0,0,0,0,0
1,2230478,1,1,0,0,0,0,0,0
2,2230529,1,1,0,0,0,0,0,0
3,2230537,1,1,0,0,0,0,0,0
4,2230579,1,1,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624963 entries, 0 to 624962
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   idPolita  624963 non-null  uint32
 1   total_ev  624963 non-null  uint8 
 2   ev_2015   624963 non-null  uint8 
 3   ev_2016   624963 non-null  uint8 
 4   ev_2017   624963 non-null  bool  
 5   ev_2018   624963 non-null  bool  
 6   ev_2019   624963 non-null  bool  
 7   ev_2020   624963 non-null  uint8 
 8   ev_2021   624963 non-null  uint8 
dtypes: bool(3), uint32(1), uint8(5)
memory usage: 7.2 MB


Three years were automatically converted into boolean values, and must be converted back to numerical.

In [6]:
df.ev_2017 = df.ev_2017.astype(np.uint8)
df.ev_2018 = df.ev_2018.astype(np.uint8)
df.ev_2019 = df.ev_2019.astype(np.uint8)

In [7]:
df.shape[0]

624963

In [8]:
# check whether the sum of the exposure years equals the column with the total number of events

df[df.total_ev != (df.ev_2015 + df.ev_2016 + df.ev_2017	+ df.ev_2018 + df.ev_2019 + df.ev_2020 + df.ev_2021)].shape[0]

21647

In [9]:
df[df.total_ev != (df.ev_2015 + df.ev_2016 + df.ev_2017	+ df.ev_2018 + df.ev_2019 + df.ev_2020 + df.ev_2021)]

,idPolita,total_ev,ev_2015,ev_2016,ev_2017,ev_2018,ev_2019,ev_2020,ev_2021
1,137,1,0,0,0,0,0,0,0
2,4570,1,0,0,0,0,0,0,0
4,11415,1,0,0,0,0,0,0,0
5,14739,1,0,0,0,0,0,0,0
6,17011,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
430241,19057448,2,0,0,0,0,1,0,0
430832,19086749,2,0,0,0,0,1,0,0
431062,19098173,2,0,0,0,0,1,0,0
431070,19098805,2,0,0,0,0,1,0,0


Some of these cells (e.g. idPolita 19146604) actually have the right number of events over the years, in the table's original version on MySQL. Therefore, it is possible that the automated downcasting, which converted 3 columns into boolean values, reduced values >1 to 1. 

Solution: reload the table from MySQL and do not perform downcasting.

In [13]:
original = db.run_view('SELECT * FROM events_no')
original[original.total_ev != (original.ev_2015 + original.ev_2016 + original.ev_2017	+ original.ev_2018 + original.ev_2019 + original.ev_2020 + original.ev_2021)]

,idPolita,total_ev,ev_2015,ev_2016,ev_2017,ev_2018,ev_2019,ev_2020,ev_2021
1,137,1,0,0,0,0,0,0,0
2,4570,1,0,0,0,0,0,0,0
4,11415,1,0,0,0,0,0,0,0
5,14739,1,0,0,0,0,0,0,0
6,17011,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
17719,2215694,1,0,0,0,0,0,0,0
17796,2218254,1,0,0,0,0,0,0,0
17799,2218475,1,0,0,0,0,0,0,0
17859,2220741,1,0,0,0,0,0,0,0


In [14]:
original[original.idPolita == 19146604]

,idPolita,total_ev,ev_2015,ev_2016,ev_2017,ev_2018,ev_2019,ev_2020,ev_2021
432031,19146604,2,0,0,0,0,2,0,0


Now there are far fewer policies with issues. The example policy (19146604) is now correct.

However, there are still 5492 incorrect rows, which may be those that were signed before the year 2015. 

The POLICIES table (polite_rca) only contains policies signed 2015 and later, where the first policy id is 2230446. I will drop all policies with ids under this number.


In [18]:
# select one such policy
original[original.idPolita < 2230446]

,idPolita,total_ev,ev_2015,ev_2016,ev_2017,ev_2018,ev_2019,ev_2020,ev_2021
0,0,1,0,0,0,1,0,0,0
1,137,1,0,0,0,0,0,0,0
2,4570,1,0,0,0,0,0,0,0
3,10662,1,0,0,1,0,0,0,0
4,11415,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
18170,2230354,2,2,0,0,0,0,0,0
18171,2230385,1,1,0,0,0,0,0,0
18172,2230389,1,1,0,0,0,0,0,0
18173,2230424,1,1,0,0,0,0,0,0


In [20]:
original.drop(original[original.idPolita < 2230446].index, axis = 0, inplace = True)
original.reset_index(drop = True, inplace = True)

In [21]:
# check the sums again
original[original.total_ev != (original.ev_2015 + original.ev_2016 + original.ev_2017	+ original.ev_2018 + original.ev_2019 + original.ev_2020 + original.ev_2021)]

,idPolita,total_ev,ev_2015,ev_2016,ev_2017,ev_2018,ev_2019,ev_2020,ev_2021


In [22]:
# save this file to S3, overwriting the previous one
original.to_feather('events_no.feather')
s3con.write('events_no.feather')
os.remove('events_no.feather')

Check whether the number of unique events from the CLAIMS table (daune) is the same as the total number of unique events from the events_no table.

In [8]:
events = s3con.read('events_no.feather')
daune = s3con.read('daune.feather')

In [14]:
# first, remove the policies from before 2015
daune.drop(daune[daune.idPolita < 2230446].index, axis = 0, inplace = True)
daune.reset_index(drop = True, inplace = True)

daune.to_feather('daune.feather')
s3con.write('daune.feather')
os.remove('daune.feather')

In [15]:
print('Number of unique events from daune:',daune.idEvent.nunique())
print('Sum of all events from events_no:', events.total_ev.sum())

Number of unique events from daune: 638826
Sum of all events from events_no: 638826


In [13]:
daune[daune.idPolita < 2230446]

,idDosar,idPolita,stare,dataDeschidere,dataAvizare,dataEveniment,tipEveniment,tipDauna,idEvent
0,53151,812068,inchis - achitat,2015-01-05,2015-01-05,2014-03-08,Pagube materiale,INTERNA,8120682014-03-08
1,53156,709184,inchis - achitat,2015-01-05,2015-01-05,2014-05-10,Pagube materiale,INTERNA,7091842014-05-10
2,53160,851329,inchis - achitat,2015-01-05,2015-01-05,2014-04-21,Pagube materiale,INTERNA,8513292014-04-21
3,53171,1529460,inchis - achitat,2015-01-05,2015-01-05,2014-07-03,Pagube materiale,INTERNA,15294602014-07-03
4,53177,1279418,inchis - achitat,2015-01-05,2015-01-05,2014-06-24,Pagube materiale,INTERNA,12794182014-06-24
...,...,...,...,...,...,...,...,...,...
704219,815819,718341,in lucru,2021-09-08,2021-09-08,2021-06-22,Pagube materiale,INTERNA,7183412021-06-22
705861,817722,725130,in lucru,2021-09-13,2021-09-13,2021-06-16,Pagube materiale,INTERNA,7251302021-06-16
706910,818925,1441335,in lucru,2021-09-15,2021-09-15,2014-09-27,"Vatamari corporale/deces, inclusive pentru pre...",INTERNA,14413352014-09-27
709003,856852,1690542,in lucru,2021-09-10,2021-09-10,2014-08-26,"Vatamari corporale/deces, inclusive pentru pre...",INTERNA,16905422014-08-26


#### Polite_exp (Exposures) was created from the CLAIMS table (daune), where the count of distinct policies was 624963.

In [3]:
exp = s3con.read('polite_exp.feather')
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598537 entries, 0 to 27598536
Data columns (total 12 columns):
 #   Column     Dtype   
---  ------     -----   
 0   idPolita   uint32  
 1   dataStart  category
 2   dataEnd    category
 3   maturity   uint8   
 4   exp_2015   category
 5   exp_2016   category
 6   exp_2017   category
 7   exp_2018   category
 8   exp_2019   category
 9   exp_2020   category
 10  exp_2021   category
 11  exp_2022   category
dtypes: category(10), uint32(1), uint8(1)
memory usage: 631.9 MB


Total number of policies here is 27,598,537 which is below the total number of policies from the attributes tables 
(27,782,469)

In [4]:
exp.head()

,idPolita,dataStart,dataEnd,maturity,exp_2015,exp_2016,exp_2017,exp_2018,exp_2019,exp_2020,exp_2021,exp_2022
0,2230446,2015-01-04,2015-07-03,6,0.4932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,2230447,2015-01-02,2015-07-01,6,0.4932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,2230448,2015-01-04,2016-01-03,12,0.9890,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,2230449,2015-01-19,2016-01-18,12,0.9479,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,2230450,2015-01-02,2016-01-01,12,0.9945,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [5]:
# check whether there are only policies from 2015 onwards
exp[exp.idPolita < 2230446]

,idPolita,dataStart,dataEnd,maturity,exp_2015,exp_2016,exp_2017,exp_2018,exp_2019,exp_2020,exp_2021,exp_2022


In [6]:
print('Number of duplicates',exp.duplicated().sum())
print('Number of nans \n', exp.isnull().sum())

Number of duplicates 5
Number of nans 
 idPolita     0
dataStart    0
dataEnd      0
maturity     0
exp_2015     0
exp_2016     0
exp_2017     0
exp_2018     0
exp_2019     0
exp_2020     0
exp_2021     0
exp_2022     0
dtype: int64
